In [44]:
import numpy as np
from tqdm import tqdm

dt = 0.1 # update time scale (ms)

################################################################################
########################## !All needed to be defined! ##########################
# Populations properties
class Population:
    NN = np.array([490, 490, 20])  # neuron numbers
    NN = np.array([10, 10, 5])  # neuron numbers
    Tag = {"dSPN":0, "iSPN":1, "FSI":2}
    Name = {0:"dSPN", 1:"iSPN", 2:"FSI"}

    # connectivity parameters of populations
    CP = np.array([[0.26, 0.27, 0.53], 
                      [0.06, 0.36, 0.36], 
                      [0, 0, 0.58]]) # connectivity probabilities
    CW = np.array([[-0.4, -1.3, -5], 
                      [-1.1, -1.1, -5], 
                      [0, 0, -0.6]]) # connectivity weigths (pA/Hz), inhibitory

    # single neuron parameters of populations
    Tau = np.array([10, 10, 10]) # time constants (ms)
    Thr = np.array([200, 200, 80]) # thresholds (pA)
    Beta = np.array([0.1, 0.1, 1]) # gains (Hz/pA)
    
    
    # LIF parameters
    Vthr = np.array([1, 1, 1])
    Vreset = np.array([0, 0, 0])
    Vrest = np.array([2, 3, 4])
########################## !All needed to be defined! ##########################
################################################################################



##########################################################
############# Connectivity Matrix (Weighted) #############
##########################################################
def create_Wmat(NN, CP, CW):
    """
    returns a random weighted connectivity matrix of several 
    neuronal populations with 'NN' as list of neuron population
    numbers, 'CP' as connectivity probability matrix and 'CW'
    as connectivity weight matrix on the population level.
    
    Examples are: 
    NN = np.array([490, 490, 20])
    CP = np.array([[0.26, 0.27, 0.53], 
                  [0.06, 0.36, 0.36], 
                  [0, 0, 0.58]]) 
    CW = np.array([[-0.4, -1.3, -5], 
                  [-1.1, -1.1, -5], 
                  [0, 0, -0.6]])  #inhibitory
    """
    nntotal = np.sum(NN) # total number of neurons
    npopulation = len(NN) # number of populations 
    
    
    Wmat = np.array([]) 
    for ind1 in range(npopulation):        
        Rowblock = np.array([])
        for ind2 in range(npopulation):
            # creating weighted connectivity matrix block 
            # of population ind2 to ind1  
            w = CW[ind1, ind2]
            Size = (NN[ind1], NN[ind2]) 
            Prob = [CP[ind1, ind2], 1-CP[ind1, ind2]] # connection probability
            Block = np.random.choice([1, 0], Size, p=Prob)*w # individual block
            Rowblock = np.concatenate((Rowblock, Block), axis=1) if Rowblock.size else Block
        Wmat = np.concatenate((Wmat, Rowblock), axis=0) if Wmat.size else Rowblock
    
    return(Wmat)


#############################################
############# Neuron Properties #############
#############################################
class Neuron:
    Tag = np.array([i for i in range(Population.NN.size) for j in range(Population.NN[i])]) # Neuron tag
    
    Tau = np.array([Population.Tau[i] for i in Tag]) # single neuron time constant (ms)
    Thr = np.array([Population.Tau[i] for i in Tag]) # threshold of transfer function (pA)
    Beta = np.array([Population.Beta[i] for i in Tag]) # gain of transfer function (Hz/pA)
    
    Vthr = np.array([Population.Vthr[i] for i in Tag]) # single neuron threshold potential in LIF
    Vreset = np.array([Population.Vreset[i] for i in Tag]) # single neuron reset potential in LIF
    Vrest = np.array([Population.Vrest[i] for i in Tag]) # single neuron rest potential in LIF
    
    Wmat = create_Wmat(Population.NN, Population.CP, Population.CW) # weighted connectivity matrix
    Cmat = np.where(Wmat!=0, 1, 0) # connectivity matrix
    
    # initial conditions
    V = np.zeros(Tag.size)
    I_out = np.zeros(Tag.size)
    V_in = np.zeros(Tag.size)




####################################
############# Dynamics #############
####################################

#### transfer function (I_in to V_in)
def phi(i_in, beta, thr):
    """
    nonlinear function used for firing rate update equation
    for input 'i_in'
    """
    if i_in>2*thr: 
        v = beta*(i_in-thr)
    else:
        v = beta*thr*np.exp((i_in-2*thr)/thr)
        
    return(v)

vphi = np.vectorize(phi) 

def vec_phi(I_in):
    global Neuron
    V_in = vphi(I_in, Neuron.Beta, Neuron.Thr)
    return(V_in)


#### single neuron model (V to I_out)
#### Leaky integrate and fire
def update_spikes(v, v_in, v_thr, v_reset, v_rest, tau):
    global dt
    i_out = 0
    if v>v_thr:
        i_out = 1
        v = v_reset
    else:
        dv = ((v_rest-v) + v_in)*dt/tau
        v += dv
    return i_out, v

vupdate_spikes = np.vectorize(update_spikes)

def update_Spikes():
    global Neuron
    (Neuron.I_out, Neuron.V) = vupdate_spikes(Neuron.V, Neuron.V_in, Neuron.Vthr, Neuron.Vreset, Neuron.Vrest, Neuron.Tau)



#### Connectivity (I_out to V_in) ####
#### Using connectivity matrix updates V and returns I_out ####
def update_Inputs():
    global Neuron
    I_in = np.matmul(Neuron.Wmat, Neuron.I_out)
    Neuron.V_in = vec_phi(I_in)

    

    
#### Updating all neurons ####
def update_Neurons():
    '''
    updates neurons voltages 'V' and spikes produced 'I_out' of 'Neuron' class.
    The update is in time interval 'dt' that should be defined outside the function.
    '''
    update_Inputs()
    update_Spikes()

    
    
#### run simulation ####    
def run(endtime, monitor_ind):
    '''
    runs a simulation of 'Neuron' with initial voltages 'V' in 'Neuron' 
    (which is a numpy array with a size of neuronnumbers), 'endtime' 
    as the final time in ms, and 'monitor_ind' as a list of indices 
    for which the spikes and voltages are recorded.
    '''
    global Neuron
    iterations = round(endtime/dt)
    monitorsize = len(monitor_ind)
    monitorV = np.zeros((monitorsize, iterations))
    monitorSpike = np.zeros((monitorsize, iterations))
    
    for ind in tqdm(range(iterations)):
        update_Neurons()
        monitorV[:, ind] = Neuron.V[monitor_ind]
        monitorSpike[:, ind] = Neuron.I_out[monitor_ind]
        
    return monitorSpike, monitorV
    

In [39]:
run(100, [13])

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 7206.68it/s]


(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [45]:
Neuron.Cmat

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
        1, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1],
       [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
        1, 1, 1],
       [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
        0, 1, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
        0, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
      